In [1]:
import numpy as np
import math
import itertools
import mosek
import ncpol2sdpa as pic
import sys

In [2]:
def K_fun(m,k,n):

    Fun=0

    for q in range(0,m+1):
        Fun+= math.comb(n-k,m-q)*math.comb(k,q)*(-1)**q
    return Fun 

In [11]:
def SDP(n,K,d,D):
    
    ##### SDP #####
    
    Ap = pic.generate_variables("Ap", n+1)
    
    sdp = pic.SdpRelaxation(Ap)
        
    L=1
    
    Eq=[]
    
    for i in range(d):
        Eq.append((K*Ap[n-i]-Ap[i]))
    
    Eq.append(Ap[0]-L)
    
    Eq.append(Ap[n]-L/K)    
    
    
    Ineq=[]
    
    for m in range(n+1):
        Ineq.append(sum((-1)**(m-j)*D**(j)*math.comb(n-j,n-m)*Ap[j] for j in range(m+1)))
    
    
    for m in range(n+1):
        Ineq.append(sum((-1)**(m-j)*D**(j)*math.comb(n-j,n-m)*(K*Ap[n-j]-Ap[j]) for j in range(m+1)))
        

    for j in range(n+1):
        Ineq.append(sum(K_fun(n-j,i,n)*Ap[i] for i in range(n+1)))    
        
    sdp.get_relaxation(1,inequalities=Ineq, equalities=Eq)
    #sdp.process_constraints(inequalities=Ineq, equalities=Eq)

    task=sdp.convert_to_mosek()
    
    
    #task.putintparam(mosek.iparam.log, 1)
    #task.putintparam(mosek.iparam.log_intpnt, 0)
    #task.putintparam(mosek.iparam.log_mio, 0)
    #task.putintparam(mosek.iparam.log_cut_second_opt, 0)
    #task.putintparam(mosek.iparam.log_sim, 0)
    #task.putintparam(mosek.iparam.log_sim_minor, 0)
    
    
    #task.set_Stream(mosek.streamtype.msg, sys.stdout.write)
    
    task.optimize()
 
    #print(task.solutionsummary(mosek.streamtype.msg))
             

    Violprimal = "{:.2e}".format(task.getsolutioninfo(mosek.soltype.itr)[1])
    Objprimal = "{:.2e}".format(task.getprimalobj(mosek.soltype.itr))
    Normprimal = "{:.2e}".format(task.getprimalsolutionnorms(mosek.soltype.itr)[2],2)
       
    solsta = task.getsolsta(mosek.soltype.itr)
    
    if solsta == mosek.solsta.optimal:
         return True
    elif solsta == mosek.solsta.dual_infeas_cer:
        print(Objprimal,Normprimal,Violprimal)
        return False
    else:
        return solsta
    
    """
    if sol.problemStatus == "feasible":
        return True                    
    elif sol.problemStatus == "infeasible":
        return False
    else:
        return sol.problemStatus 
    
    """

D=2

Codes=[
[1,0,1],[1,1,1],
[2,0,2],[2,1,1],[2,2,1],
[3,0,2],[3,1,1],[3,2,1],[3,3,1],
[4,0,2],[4,1,2],[4,2,2],[4,3,1],[4,4,1],
[5,0,3],[5,1,3],[5,2,2],[5,3,1],[5,4,1],[5,5,1],
[6,0,4],[6,1,3],[6,2,2],[6,3,2],[6,4,2],[6,5,1],[6,6,1],
[7,0,3],[7,1,3],[7,2,2],[7,3,2],[7,4,2],[7,5,1],[7,6,1],[7,7,1],
]

Codes=[
[1,0,1],[1,1,1],
[2,0,2],[2,1,1],[2,2,1],
[3,0,2],[3,1,1],[3,2,1],[3,3,1],
[4,0,2],[4,1,2],[4,2,2],[4,3,1],[4,4,1],
[5,0,3],[5,1,3],[5,2,2],[5,3,1],[5,4,1],[5,5,1],
[6,0,4],[6,1,3],[6,2,2],[6,3,2],[6,4,2],[6,5,1],[6,6,1],
[7,0,3],[7,1,3],[7,2,2],[7,3,2],[7,4,2],[7,5,1],[7,6,1],[7,7,1],

[8,0,4],[8,1,3],[8,2,3],[8,3,3],[8,4,2],[8,5,2],[8,6,2],[8,7,1],[8,8,1],
[9,0,4],[9,1,3],[9,2,3],[9,3,3],[9,4,2],[9,5,2],[9,6,2],[9,7,1],[9,8,1],[9,9,1],
[10,0,4],[10,1,4],[10,2,4],[10,3,3],[10,4,3],[10,5,2],[10,6,2],[10,7,2],[10,8,2],[10,9,1],[10,10,1]
]

#### Existing qubit codes ####

for code in Codes:
    
    n,K,d=code

    check=SDP(n,2**K,d,D)
    
    if check != True:
        print("%s=%s" % (code,check))

print("Done")


#### Not existing qubit codes (distance+1) ####

for code in Codes:
    
    n,K,d=code
    p=n
    check=SDP(n,2**K,d+1,D)
    
    if check != False:
        print("[%s,%s,%s]=%s" % (code[0],code[1],code[2]+1,check))

print("Done")


Done
[1,0,2]=True
-4.97e-01 3.39e+00 6.70e-13
[2,0,3]=True
-4.42e-01 4.98e+00 1.63e-12
-5.97e-01 3.00e+00 5.13e-12
[3,0,3]=True
-1.85e-01 5.64e+00 1.36e-13
-3.92e-01 4.11e+00 6.03e-13
-4.70e-01 4.59e+00 2.11e-13
[4,0,3]=True
-4.65e-01 6.45e+00 3.68e-13
-5.35e-01 5.25e+00 6.50e-13
-3.11e-01 5.05e+00 6.02e-12
-4.06e-01 5.15e+00 8.17e-12
[5,0,4]=True
-5.30e-01 5.87e+00 5.61e-12
-4.01e-01 6.16e+00 6.43e-12
-6.92e-02 1.04e+01 8.94e-13
-2.66e-01 5.23e+00 3.24e-12
-3.37e-01 5.37e+00 2.51e-12
[6,0,5]=True
-4.86e-01 7.17e+00 4.09e-11
-1.98e-01 5.36e+00 1.05e-11
-3.72e-01 6.84e+00 2.08e-12
-4.51e-01 7.34e+00 6.93e-12
-2.32e-01 5.34e+00 1.67e-10
-3.01e-01 5.47e+00 6.99e-11
[7,0,4]=True
-3.03e-01 7.04e+00 9.93e-11
-3.92e-02 5.87e+00 1.36e-11
-2.37e-01 6.56e+00 1.08e-11
-3.61e-01 7.52e+00 6.32e-10
-1.72e-02 1.64e+01 2.05e-10
-2.01e-01 5.45e+00 1.72e-10
-2.70e-01 5.56e+00 8.13e-10
[8,0,5]=True
-2.35e-01 6.45e+00 5.37e-11
-3.21e-01 7.11e+00 6.47e-11
-4.33e-01 9.88e+00 1.23e-10
-2.34e-01 7.35e+00 2.36

In [7]:
D=2

Codes=[[7,26,2],[9,112,2],[11,460,2],[13,1877,2],[15,7606,2]]

for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],codes[1],codes[2],D)))

[7, 26, 2]=True
[9, 112, 2]=True
[11, 460, 2]=True
[13, 1877, 2]=True
[15, 7606, 2]=True


In [8]:
D=2


Codes=[[7,3,3],[8,9,3],[9,13,3],[11,53,3],[12,89,3],[13,204,3],[14,324,3],[15,580,3]]


for codes in Codes:

    print("%s=%s" % (codes,SDP(codes[0],codes[1],codes[2],D)))

D=2


Codes=[[7,1,4],[10,5,4],[11,7,4],[12,20,4],[13,40,4],[14,102,4],[15,150,4]]

    

for codes in Codes:

    print("%s=%s" % (codes,SDP(codes[0],codes[1],codes[2],D)))
    
D=2

Codes=[[13,3,5],[14,10,5],[15,18,5]]
    
for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],codes[1],codes[2],D)))
    
Codes=[[14,3,5],[24,0,10]]

for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],2**codes[1],codes[2],D)))

[7, 3, 3]=True
[8, 9, 3]=True
[9, 13, 3]=True
[11, 53, 3]=True
[12, 89, 3]=True
[13, 204, 3]=True
[14, 324, 3]=True
[15, 580, 3]=True
[7, 1, 4]=True
[10, 5, 4]=True
[11, 7, 4]=True
[12, 20, 4]=True
[13, 40, 4]=True
[14, 102, 4]=True
[15, 150, 4]=True
[13, 3, 5]=True
[14, 10, 5]=True
[15, 18, 5]=True
[14, 3, 5]=True
[24, 0, 10]=solsta.prim_illposed_cer
